# KNN

In [16]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import ast

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

Step 1: Load dataset

In [2]:
url_mov='https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'
url_cred='https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'

movies = pd.read_csv(url_mov)
credits = pd.read_csv(url_cred)

Step 2: Explore both dataset

In [3]:
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
867,54000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",http://www.imdb.com/title/tt0099674/,242,"[{""id"": 131, ""name"": ""italy""}, {""id"": 186, ""na...",en,The Godfather: Part III,In the midst of trying to legitimize his busin...,59.194915,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""IT"", ""name"": ""Italy""}, {""iso_...",1990-12-24,136766062,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,All the power on earth can't change destiny.,The Godfather: Part III,7.1,1546
294,100000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 12, ""...",NaN,116711,"[{""id"": 2486, ""name"": ""fantasy""}, {""id"": 15907...",en,Epic,A teenager finds herself transported to a deep...,36.711378,"[{""name"": ""Blue Sky Studios"", ""id"": 9383}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-05-15,268426634,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Discover a world beyond your imagination,Epic,6.4,1121
1662,18000000,"[{""id"": 10752, ""name"": ""War""}]",NaN,9665,"[{""id"": 6091, ""name"": ""war""}, {""id"": 12425, ""n...",en,Glory,Robert Gould Shaw leads the US Civil War's fir...,16.102271,"[{""name"": ""TriStar Pictures"", ""id"": 559}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1989-12-15,26828365,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Their innocence. Their heritage. Their lives.,Glory,7.4,383
2040,30000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",NaN,2176,"[{""id"": 387, ""name"": ""california""}, {""id"": 115...",en,The Glass House,After the parents of Ruby and her younger brot...,9.448517,"[{""name"": ""Original Film"", ""id"": 333}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-09-14,23619609,106.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, ...",Released,The glass is about to be shattered...,The Glass House,5.4,201
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2010-11-24,591794936,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330


In [4]:
credits.sample(5)

,movie_id,title,cast,crew
3480,8545,Little Voice,"[{""cast_id"": 4, ""character"": ""Mari Hoff"", ""cre...","[{""credit_id"": ""56e7eae19251410a3a000b5f"", ""de..."
4654,24869,Ink,"[{""cast_id"": 3, ""character"": ""John"", ""credit_i...","[{""credit_id"": ""52fe44adc3a368484e030a09"", ""de..."
137,49444,Kung Fu Panda 2,"[{""cast_id"": 5, ""character"": ""Po (voice)"", ""cr...","[{""credit_id"": ""57deba7cc3a36813080070c8"", ""de..."
3504,9282,11:14,"[{""cast_id"": 10, ""character"": ""Jack"", ""credit_...","[{""credit_id"": ""52fe44e1c3a36847f80af88d"", ""de..."
2463,318850,The Young Messiah,"[{""cast_id"": 12, ""character"": ""Jesus"", ""credit...","[{""credit_id"": ""58cd46d0c3a36850e900c1bd"", ""de..."


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


Step 3: Merge both dataset

In [7]:
movies = movies.merge(credits, right_on=['movie_id','title'],left_on=['id','title']).drop('movie_id',axis=1)

In [8]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

Step 4: select the variables.

In [10]:
movies = movies[['id','title','overview','genres','keywords','cast','crew']]

Step 5: explore the final variables

In [11]:
movies.sample(5)

,id,title,overview,genres,keywords,cast,crew
179,270946,Penguins of Madagascar,"Skipper, Kowalski, Rico and Private join force...","[{""id"": 10751, ""name"": ""Family""}, {""id"": 16, ""...","[{""id"": 3028, ""name"": ""penguin""}, {""id"": 3645,...","[{""cast_id"": 8, ""character"": ""Skipper (voice)""...","[{""credit_id"": ""5483ba4d92514175f5002c15"", ""de..."
350,4327,Charlie's Angels,Aspects of this take on the 1970s hit TV serie...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 779, ""name"": ""martial arts""}, {""id"": 5...","[{""cast_id"": 1, ""character"": ""Natalie Cook"", ""...","[{""credit_id"": ""52fe43bac3a36847f806b227"", ""de..."
4239,39541,Stolen Summer,"Pete, an eight-year-old Catholic boy growing u...","[{""id"": 18, ""name"": ""Drama""}, {""id"": 10751, ""n...",[],"[{""cast_id"": 3, ""character"": ""Joe O'Malley"", ""...","[{""credit_id"": ""52fe47249251416c9106b639"", ""de..."
4552,117942,Girls Gone Dead,A group of six ex-high school cheerleaders are...,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 35, ""nam...",[],"[{""cast_id"": 24, ""character"": ""Rebecca Foster""...","[{""credit_id"": ""52fe4bc4c3a36847f821312d"", ""de..."
2399,175541,Black Nativity,A street-wise teen from Baltimore who has been...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10402, ""n...","[{""id"": 4344, ""name"": ""musical""}, {""id"": 15837...","[{""cast_id"": 2, ""character"": ""Reverend Cornell...","[{""credit_id"": ""52fe4d60c3a36847f825d973"", ""de..."


In [12]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4803 non-null   int64 
 1   title     4803 non-null   object
 2   overview  4800 non-null   object
 3   genres    4803 non-null   object
 4   keywords  4803 non-null   object
 5   cast      4803 non-null   object
 6   crew      4803 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.2+ KB


In [13]:
movies.dropna(inplace=True)
movies.reset_index(inplace=True)

Step 6: dataset transform

In [14]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [17]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [18]:
def convert3(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
        count +=1  
    return L

In [19]:
movies['cast'] = movies['cast'].apply(convert3)

In [20]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [21]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [23]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

Step 7: remove the space from all the strings except title

In [25]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [26]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

Step 8: combine columns into one column named 'tags'

In [27]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [28]:
new_df = movies[['id','title','tags']].copy()

new_df['tags'] = new_df['tags'].apply(lambda x :" ".join(x))

Step 9: tokenization of column 'tags'

In [29]:
cv = CountVectorizer(max_features=5000 ,stop_words='english')

In [31]:
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors.shape

(4800, 5000)

In [32]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

Step 10: compare

In [33]:
cosine_similarity(vectors).shape

(4800, 4800)

In [34]:
similarity = cosine_similarity(vectors)

In [35]:
similarity[0] # similarity between the label of the first film and the other films, one by one. 

array([1.        , 0.08980265, 0.05986843, ..., 0.02457366, 0.02777778,
       0.        ])

In [36]:
print('The six tags with the greatest similarity to the first are:')
sorted(list(enumerate(similarity[0])),reverse =True , key = lambda x:x[1])[1:6]

The six tags with the greatest similarity to the first are:


[(539, 0.25724787771376323),
 (1191, 0.2545875386086578),
 (507, 0.24948506639458295),
 (1213, 0.24595492912420727),
 (260, 0.2434322477800738)]

Step 11: Finally, create a recommendation function based on the cosine_similarity. This function should recommend the 5 most similar movies.

In [37]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0] ##fetching the movie index
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate( distances)),reverse =True , key = lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

Step 12: Check your recommender system by introducing a movie. Run to see the recommendations.

In [39]:
recommend('Silver Medalist')

30 Minutes or Less
The Helix... Loaded
Jonah: A VeggieTales Movie
Small Soldiers
MacGruber


In [41]:
recommend('Epic')

Mirrormask
Rise of the Guardians
Return to Never Land
Foodfight!
The Brave Little Toaster
